In [29]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import os

import requests
from PIL import Image
from io import BytesIO
from IPython.display import display

from typing import List
import torch

import chromadb

import shutil

import os
import datetime

from transformers import ViTImageProcessor, ViTModel, ViTImageProcessorFast
from PIL import Image
import requests

import pandas as pd
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.utils.data_loaders import ImageLoader

import matplotlib.pyplot as plt

In [30]:
# Set the path to the file you'd like to load
file_name = "test_images_vector_df_csv.csv"
file_path = "/Users/shogun/code/gwen-m97/raw_data/test_images/"

# Load the latest version
test_image_vector_df = pd.read_csv(f"{file_path}{file_name}")

test_image_vector_df.head()

,Unnamed: 0,image,image_vector
0,0,Paul_Cézanne_-_The_Basket_of_Apples_-_1926.252...,[-1.9647181e-05 1.3060389e-02 -2.9653966e-01 ...


In [31]:
# Set the path to the file you'd like to load
file_name = "gwen_200_vector_df_csv_first.csv"
file_path = "/Users/shogun/code/gwen-m97/raw_data/gwen_200/"

# Load the latest version
gwen_200_vector_df_first = pd.read_csv(f"{file_path}{file_name}")

gwen_200_vector_df_first.head()

,Unnamed: 0,image,image_vector
0,0,fred,john
1,1,pierre-paul-prud-hon_louis-antoine-de-saint-ju...,"tensor([[[ 0.0779, -0.0746, -0.0250, ..., -0...."
2,2,pierre-paul-prud-hon_study-for-venus-and-adoni...,"tensor([[[ 0.0945, -0.1561, -0.1269, ..., -0...."
3,3,pierre-paul-prud-hon_untitled.jpg,"tensor([[[-0.1026, -0.0752, 0.0328, ..., -0...."
4,4,pyotr-konchalovsky_crimea-the-rocks-by-the-sea...,"tensor([[[-0.0983, -0.1852, -0.0368, ..., 0...."


In [32]:
# Set the path to the file you'd like to load
file_name = "gwen_200_vector_df_csv_second.csv"
file_path = "/Users/shogun/code/gwen-m97/raw_data/gwen_200/"

# Load the latest version
gwen_200_vector_df_second = pd.read_csv(f"{file_path}{file_name}")

gwen_200_vector_df_second.head()

,Unnamed: 0,image,image_vector
0,0,fred,john
1,1,abbott-handerson-thayer_blue-jays-in-winter-19...,"tensor([[[ 0.0911, -0.1870, -0.0223, ..., -0...."
2,2,albert-gleizes_untitled-2.jpg!Large.jpg,"tensor([[[-0.0461, -0.1976, -0.0997, ..., 0...."
3,3,alexander-shilov_rosemary-flowered-1980(4).jpg...,"tensor([[[ 0.0573, -0.0742, -0.0356, ..., -0...."
4,4,alexandre-jacovleff_nu-2.jpg!Large.jpg,"tensor([[[ 1.8831e-01, -2.9391e-01, 2.2604e-0..."


In [33]:
gwen_200_vector_df = pd.concat([gwen_200_vector_df_first,gwen_200_vector_df_second],ignore_index=True)
gwen_200_vector_df.head()

,Unnamed: 0,image,image_vector
0,0,fred,john
1,1,pierre-paul-prud-hon_louis-antoine-de-saint-ju...,"tensor([[[ 0.0779, -0.0746, -0.0250, ..., -0...."
2,2,pierre-paul-prud-hon_study-for-venus-and-adoni...,"tensor([[[ 0.0945, -0.1561, -0.1269, ..., -0...."
3,3,pierre-paul-prud-hon_untitled.jpg,"tensor([[[-0.1026, -0.0752, 0.0328, ..., -0...."
4,4,pyotr-konchalovsky_crimea-the-rocks-by-the-sea...,"tensor([[[-0.0983, -0.1852, -0.0368, ..., 0...."


In [34]:
gwen_200_vector_df.shape

(195, 3)

In [35]:
gwen_200_vector_df.dtypes

Unnamed: 0       int64
image           object
image_vector    object
dtype: object

In [36]:
gwen_200_vector_df['image_vector'].to_numpy()[1].rsplit("tensor(")[1].rsplit(",\n")[0].to_numpy()

AttributeError: 'str' object has no attribute 'to_numpy'

In [37]:
class GoogleVITHuge224Embedding(EmbeddingFunction):


    def __call__(self, input: Documents) -> Embeddings:

        from transformers import ViTImageProcessor, ViTModel, ViTImageProcessorFast

        #Instantiate the image. Convert it to 244 x 244 and normalise RGB between 0 and 1 witha mean of 0.5 for each channel

        self.feature_extractor = ViTImageProcessorFast.from_pretrained('google/vit-huge-patch14-224-in21k')

        #Instantiate the Google ViT with pretrained weights

        self.model = ViTModel.from_pretrained('google/vit-huge-patch14-224-in21k')#Preprocess the data

        inputs = self.feature_extractor(images=input, return_tensors="pt")

        #Embedd the data

        outputs = self.model(**inputs)

        #Convert the embedding to a Numpy array and take the first vector of the Transformer state

        embeddings = outputs.last_hidden_state.data.numpy()[0,0]

        print(embeddings)

        #return the embedding

        return embeddings

In [38]:
chroma_client = chromadb.PersistentClient(path='/Users/shogun/code/gwen-m97/inspiart/my_vectordb',settings=Settings(allow_reset=True))


NameError: name 'Settings' is not defined

In [39]:
image_loader = ImageLoader()

image_embbeding_function = GoogleVITHuge224Embedding()


/var/folders/1r/626773hx3mbf2w_31tdjwv6m0000gn/T/ipykernel_16312/3517319003.py:3: DeprecationWarning: The class GoogleVITHuge224Embedding does not implement __init__. This will be required in a future version.
  image_embbeding_function = GoogleVITHuge224Embedding()


In [40]:
images_db = chroma_client.get_or_create_collection(name="images_db", embedding_function=image_embbeding_function, data_loader=image_loader)

InternalError: Database error: error returned from database: (code: 14) unable to open database file

In [41]:
query_uris = '/Users/shogun/code/gwen-m97/raw_data/test_images/Two_Young_Girls_at_the_Piano_MET_rl1975.1.201.R.jpg'

In [42]:
image_suggestions = images_db.query(
    query_uris=[query_uris]
)

[-0.00713051 -0.12309302  0.02648428 ... -0.09540542  0.10423515
 -0.03038444]


InternalError: Error getting collection: Database error: error returned from database: (code: 14) unable to open database file

In [8]:
image_suggestions

{'ids': [['7731',
   '5722',
   '7754',
   '7741',
   '7753',
   '5719',
   '8180',
   '7539',
   '8010',
   '7743']],
 'embeddings': None,
 'documents': [[None, None, None, None, None, None, None, None, None, None]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'image': 'correggio_adoration-of-the-child-1520(2).jpg',
    'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_adoration-of-the-child-1520(2).jpg'},
   {'image': 'cima-da-conegliano_the-deposition.jpg',
    'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/cima-da-conegliano_the-deposition.jpg'},
   {'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_madonna-and-child-1514(1).jpg',
    'image': 'correggio_madonna-and-child-1514(1).jpg'},
   {'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_danae-1531(2).jpg',
    'image': 'correggio_danae-1531(2).jpg'},
   {'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_ma

In [9]:
def print_query_results(query_image, query_results: dict)->None:
    result_count = len(query_results['ids'][0])
    print(result_count)

    query_image_path = query_image
    query_image_pil = Image.open(query_image_path)
    query_image_pil.show()

    for j in range(result_count):
        id       = query_results["ids"][0][j]
        distance = query_results['distances'][0][j]
        data     = query_results['data']
        document = query_results['documents'][0][j]
        metadata = query_results['metadatas'][0][j]
        uri      = query_results['uris']

        print(f'id: {id}, distance: {distance}, metadata: {metadata}, document: {document}')

        # Display image, the physical file must exist at URI.
        # (ImageLoader loads the image from file)

        image_path = query_results['metadatas'][0][j]['url']
        image_pil = Image.open(image_path)
        image_pil.show()

In [10]:
result_count = len(image_suggestions['ids'][0])

In [11]:
result_count

10

In [108]:
image_suggestions['metadatas'][0][0]['url']


'/Users/shogun/code/gwen-m97/raw_data/Pop_Art/patrick-caulfield_bathroom-mirror-1968.jpg'

In [22]:
print_query_results(query_uris,image_suggestions)

10
id: 7731, distance: 4.267673492431641, metadata: {'image': 'correggio_adoration-of-the-child-1520(2).jpg', 'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_adoration-of-the-child-1520(2).jpg'}, document: None
id: 5722, distance: 4.325127601623535, metadata: {'image': 'cima-da-conegliano_the-deposition.jpg', 'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/cima-da-conegliano_the-deposition.jpg'}, document: None
id: 7754, distance: 4.404486179351807, metadata: {'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_madonna-and-child-1514(1).jpg', 'image': 'correggio_madonna-and-child-1514(1).jpg'}, document: None
id: 7741, distance: 4.4378132820129395, metadata: {'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_danae-1531(2).jpg', 'image': 'correggio_danae-1531(2).jpg'}, document: None
id: 7753, distance: 4.4475579261779785, metadata: {'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_madonna-1514(2).jpg', 'image': 'correg

In [12]:
images_db.count()

8035

In [21]:
chroma_client.heartbeat()

1756896263216687000